In [46]:
%load_ext autoreload

%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 999


import time
import unicodedata

import datetime
from datetime import timedelta, datetime

import csv
import os

import sys
sys.path.append('./lib/')
from functions import *
from tqdm import tnrange, tqdm_notebook
from unidecode import unidecode

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import glob

%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
sns.set_style("darkgrid")

import locale
locale.setlocale(locale.LC_ALL, 'es_ES')

'es_ES'

# Import data

In [3]:
period = '20180210-3'

In [9]:
df = pd.read_csv('../data/output/scraped_data_clean_{}.csv'.format(period), encoding='utf-8', low_memory=False)

In [10]:
df.shape

(1954998, 23)

# Clean dates 

In [136]:
pd_preprocess(df)

### Reset all fixing

In [137]:
del df['datets']

### Fixing -allyear 

In [138]:
pd_runfix(pd_allyear, df, 'datets', 'yearmonth')

----BEFORE FIX ------
Coverage before fix:
No date timestamp column. Creating
False    1954998
Name: datets, dtype: int64
----AFTER FIX ------
Coverage before fix:
False    1500250
True      454748
Name: datets, dtype: int64
Improved: False   -454748.0
True          NaN
Name: datets, dtype: float64


### Year - Month

In [139]:
pd_runfix(pd_yearmonth, df, 'datets', 'yearmonth')

----BEFORE FIX ------
Coverage before fix:
False    1500250
True      454748
Name: datets, dtype: int64
----AFTER FIX ------
Coverage before fix:
True     1779846
False     175152
Name: datets, dtype: int64
Improved: False   -1325098
True     1325098
Name: datets, dtype: int64


### month-allyear 

In [140]:
pd_runfix(pd_monthallyear, df, 'datets', 'yearmonth')

----BEFORE FIX ------
Coverage before fix:
True     1779846
False     175152
Name: datets, dtype: int64
----AFTER FIX ------
Coverage before fix:
True     1924735
False      30263
Name: datets, dtype: int64
Improved: True     144889
False   -144889
Name: datets, dtype: int64


### year month 2

In [141]:
pd_runfix(pd_yearmonth2, df, 'datets', 'yearmonth')

----BEFORE FIX ------
Coverage before fix:
True     1924735
False      30263
Name: datets, dtype: int64
----AFTER FIX ------
Coverage before fix:
True     1924735
False      30263
Name: datets, dtype: int64
Improved: True     0
False    0
Name: datets, dtype: int64


### Drop aux columns 

In [156]:
cols = [x for x in df.columns if 'aux' in x]
cols

['aux', 'aux2', 'aux3']

In [157]:
for c in cols:
    del df[c]

### what's left

In [142]:
df.loc[pd.isnull(df['datets']), 'yearmonth'].value_counts().sum()

30263

In [143]:
df.loc[pd.isnull(df['datets']), 'yearmonth'].value_counts()

2015-dotaciónacontrata2015                            7935
2016-2012                                             1189
verhistórico-2010                                     1017
2013-dotaciónacontrata2013                             974
2015-dotacióndeplanta2015                              830
verhistórico-2012                                      717
2014-dotaciónacontrata2014                             714
verhistórico-2011                                      691
verhistórico-2013                                      672
verhistórico-2016                                      663
2016-2014                                              661
2009(enero-abril)-allyear                              657
verhistórico-2015                                      626
verhistórico-2014                                      617
2016-historico                                         609
2016-2013                                              550
2012-dotaciónacontrata2012                             5

# Print out file

In [158]:
df.to_csv('../data/output/clean_dates_{}.csv'.format(period), encoding='utf-8', sep='|', quoting=csv.QUOTE_ALL)

# Cleanup stats 

In [155]:
df['pd'] = df['datets'].isnull()
a = df.groupby(['entity', 'department', 'pd'])['entity'].count().unstack('pd')
a.columns = ['Data', 'noData']
a.sort_values('noData', ascending=False)



Data  \
entity                                          department                                                     
Ministerio de Educación                         Superintendencia de Educación                         6070.0   
Ministerio del Interior y Seguridad Pública     Gobierno Regional de Los Ríos                         1032.0   
Ministerio de Vivienda y Urbanismo              SERVIU III Región                                     5348.0   
Ministerio de Hacienda                          Dirección de Presupuestos (DIPRES)                     423.0   
Ministerio del Trabajo y Previsión Social       Subsecretaría del Trabajo                            10946.0   
                                                Subsecretaría de Previsión Social                        NaN   
Ministerio de Energía                           Comisión Nacional de Energía (CNE)                       NaN   
Ministerio de Desarrollo Social                 Subsecretaría de Planificación                       13697.0   
Ministerio de Defensa Nacional                  Caja de Previsión de la Defensa Nacional (CAPRE...   14133.0   
Ministerio de Obras Públicas                    Dirección General de Aguas (DGA)                     43050.0   
Ministerio de Desarrollo Social                 Instituto Nacional de la Juventud (INJUV)             8006.0   
Ministerio de Salud                             per_planta                                               NaN   
Ministerio de Educación                         Subsecretaría de Educación Parvularia                  163.0   
Consejo Nacional de Televisión                  Consejo Nacional de Televisión (CNTV)                 2170.0   
Consejo Nacional de la Cultura y las Artes      Consejo Nacional de la Cultura y las Artes            3436.0   
Consejo de Defensa del Estado                   Consejo de Defensa del Estado                        56047.0   
                                                per_contrata                                          1192.0   
                                                per_planta                                             728.0   
Ministerio Secretaría General de Gobierno       Subsecretaría General de Gobierno                     5159.0   
Ministerio Secretaría General de la Presidencia Subsecretaría General de la Presidencia              14657.0   
Ministerio de Agricultura                       Instituto de Desarrollo Agropecuario (INDAP)         14379.0   
                                                Oficina de Estudios y Políticas Agrarias (ODEPA)      1576.0   
                                                Servicio Agrícola Ganadero (SAG)                     27606.0   
                                                Subsecretaría de Agricultura                          1030.0   
Ministerio de Defensa Nacional                  Subsecretaría de Defensa                              4740.0   
                                                Subsecretaría de Investigaciones                        58.0   
                                                Subsecretaría para las Fuerzas Armadas               27887.0   
Ministerio de Desarrollo Social                 Corporación Nacional de Desarrollo Indígena (CO...    5041.0   
                                                Servicio Nacional del Adulto Mayor (SENAMA)          12249.0   
                                                Subsecretaría de Evaluación Social                   16883.0   
                                                Subsecretaría de Servicios Sociales                  49452.0   
                                                per_contrata                                           405.0   
                                                per_planta                                             285.0   
Ministerio de Economía, Fomento y Turismo       Comité de Inversiones Extranjeras                      336.0   
                                                Fiscalía Nacional Económ